# Data cleaning

This script:

* extracts the reviews from each file in the data directory (default: "./data/raw_data/movie_reviews")
* cleans up data in each file:
  * convert critics_pick to a boolean type
  * remove unnecessary columns ('link' and 'multimedia')
  * replaces opening dates of '0000-00-00' with None
  * convert date columns to datetime type
  * remove newlines and carriage returns from the summary
* combines them into a single data frame
* saves the combined data frame to the output file (default: "/cleaned_data/movie_reviews.csv").

In [2]:
import numpy as np
import pandas as pd
import json
import os


# Path to the data directory into which downloaded JSON is saved.
data_path = os.path.join(".", "data", "raw_data", "movie_reviews")
print(data_path)

if not os.path.exists(data_path):
    print("Creating data directory: {}".format(data_path))
    os.makedirs(data_path)

if os.path.isdir(data_path):
    print(data_path + " is a directory")
else:
    print(data_path + " is NOT a directory - something is wrong :(")

    
# Path to the output directory.
csv_file_path = os.path.join(".", "cleaned_data", "movie_reviews.csv")

if not os.path.exists(csv_file_path):
    print("Creating data directory: {}".format(csv_file_path))
    os.makedirs(csv_file_path)

.\data\raw_data\movie_reviews
.\data\raw_data\movie_reviews is a directory


In [3]:
# Utility function to get the list of raw data JSON file paths.
#
# d - directory to search in
# suffix - only return files ending with this suffix, defaults to ".json"
# returns a list of files 
def get_raw_json_file_paths(d, suffix='.json'):
    
    filelist = [f for f in os.listdir(d) if 
                os.path.isfile(os.path.join(d, f)) and f.endswith(suffix)]
    
    return filelist

In [4]:
# Utility function to restore the datetime type to date columns.
#
# df - dataframe to adjust
# returns dataframe with date columns as datetime types
def restore_date_columns(df):
    # Convert dates from object/string into datetime.
    df['publication_date'] = pd.to_datetime(df['publication_date'])
    df['date_updated'] = pd.to_datetime(df['date_updated'])

    # opening_date has some 'interesting' data (e.g. 1998-00-00) so leave that alone for now.
    #df['opening_date'] = pd.to_datetime(df['opening_date'])

    # Clean up any dates with the value "0000-00-00".
    df.loc[df['opening_date'] == "0000-00-00", ['opening_date']] = None
    
    return df
    

# Function to create a dataframe from a raw data file.
#
# The raw data file includes metadata from the server; this is excluded from the dataframe.
#
# f - path to raw data file
# returns dataframe created from results in raw data file
def create_data_frame(f, verbose=False):
    if verbose:
        print("create_data_frame() - filename: {}".format(f))
    
    # Read the object.
    reviews = None
    with open(f, "rt", encoding="UTF-8") as json_data:
        reviews = json.load(json_data)
        if verbose:
            print(reviews)

    reviews = reviews['results']
    if verbose:
        print(reviews)
    
    # Turn it into a data frame.
    result = pd.DataFrame(reviews)
    if verbose:
        print(result.shape)
        print(result.head(20))

    
    # Convert critics_pick from float to boolean.
    result['critics_pick'] = result['critics_pick'].astype(bool)

    # Punctuation is inconsistent in this field, so remove it.
    result['byline'] = result['byline'].str.replace('.', ' ')
    result['byline'] = result['byline'].str.replace('-', ' ')
    result['byline'] = result['byline'].str.replace('_', ' ')
    result['byline'] = result['byline'].str.replace('%', ' ')
    result['byline'] = result['byline'].str.replace(':', ' ')
    result['byline'] = result['byline'].str.replace(']', ' ')
    result['byline'] = result['byline'].str.replace('[', ' ')

    # Some bylines end with a comma; remove it.
    result.loc[result['byline'].str.endswith(','), ['byline']] = result['byline'].str[:-1]
    
    # Collapse runs of whitespace.
    result['byline'] = result['byline'].str.replace('    ', ' ')
    result['byline'] = result['byline'].str.replace('   ', ' ')
    result['byline'] = result['byline'].str.replace('  ', ' ')

    # Remove leading and trailing whitespace.
    result['byline'] = result['byline'].str.strip()

    # Bylines are mixed case, so convert to title case.
    result['byline'] = result['byline'].str.title()

    # These poor fellows have their names misspelled terribly.
    result.loc[result['byline'] == "Moroaunt Hall", ['byline']] = "Mordaunt Hall"
    result.loc[result['byline'] == "Mokdaunt Hall", ['byline']] = "Mordaunt Hall"
    result.loc[result['byline'] == "Modaunt Hall", ['byline']] = "Mordaunt Hall"
    result.loc[result['byline'] == "Modaut Hall", ['byline']] = "Mordaunt Hall"
    result.loc[result['byline'] == "Mordaut Hall", ['byline']] = "Mordaunt Hall"

    result.loc[result['byline'] == "Bosuey Crowther", ['byline']] = "Bosley Crowther"
    result.loc[result['byline'] == "Bossley Crowther", ['byline']] = "Bosley Crowther"
    result.loc[result['byline'] == "Bosley Chowther", ['byline']] = "Bosley Crowther"
    
    result.loc[result['byline'] == "Andrbe Sennwald", ['byline']] = "Andre Sennwald"
    result.loc[result['byline'] == "Andre Senn Wald", ['byline']] = "Andre Sennwald"

    
    # Some summaries end in newline characters, some contain carriage returns and newlines.
    result['summary_short'] = result['summary_short'].str.strip()
    result['summary_short'] = result['summary_short'].str.replace('\r', '')
    result['summary_short'] = result['summary_short'].str.replace('\n', '')

    
    # Convert dates from object/string into datetime.
    result = restore_date_columns(result)
    
    # Drop link (URL to full review) and multimedia (image) since we don't care about those.
    return result.drop(['link', 'multimedia'], axis=1)

In [5]:
# Function to create a combined CSV file from a list of dataframes
#
# df_list - list of dataframes to combine and write
# outfilepath - path to the CSV file to write
# returns nothing
def create_combined_csv(df_list, outfilepath):
    
    combined_df = pd.concat(df_list)
    print(combined_df.shape)
    
    # QUOTE_NONNUMERIC (2) because we expect commas to be present in text fields.
    combined_df.to_csv(outfilepath, index=False, quoting=2, encoding="UTF-8")

In [6]:
raw_data_file_names = get_raw_json_file_paths(data_path)
dataframes = [create_data_frame(os.path.join(data_path, f), verbose=False) for f in raw_data_file_names]
print("Loaded {} dataframes".format(len(dataframes)))

Loaded 1345 dataframes


In [7]:
# Write the combined CSV file.
create_combined_csv(dataframes, csv_file_path)

# And read it back in.
restored_df = pd.read_csv(csv_file_path, quoting=2, parse_dates=True, infer_datetime_format=True, encoding="UTF-8")
restored_df = restore_date_columns(restored_df)

print(restored_df.shape)
print(restored_df.info())
print(restored_df.head(20))

(26893, 9)
(26893, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26893 entries, 0 to 26892
Data columns (total 9 columns):
byline              26893 non-null object
critics_pick        26893 non-null bool
date_updated        26878 non-null datetime64[ns]
display_title       26893 non-null object
headline            26893 non-null object
mpaa_rating         13516 non-null object
opening_date        16971 non-null object
publication_date    26893 non-null datetime64[ns]
summary_short       20764 non-null object
dtypes: bool(1), datetime64[ns](2), object(6)
memory usage: 1.7+ MB
None
                 byline critics_pick        date_updated  \
0        Manohla Dargis         True 2017-04-18 16:44:22   
1             A O Scott         True 2017-04-18 16:44:21   
2             A O Scott         True 2017-04-18 16:44:21   
3   Jeannette Catsoulis        False 2017-04-18 16:44:26   
4           Glenn Kenny        False 2017-04-13 21:58:02   
5   Jeannette Catsoulis        False 2017-04-

In [8]:
restored_df.describe()

,byline,critics_pick,date_updated,display_title,headline,mpaa_rating,opening_date,publication_date,summary_short
count,26893,26893,26878,26893,26893,13516,16971,26893,20764
unique,301,2,3359,25744,26656,7,8064,12692,20718
top,Bosley Crowther,False,2016-09-23 12:44:01,Hamlet,Devotion,R,2014-01-10,2013-10-17 00:00:00,Routine.
freq,3022,23766,16,7,4,5948,16,25,6
first,NaN,NaN,2016-03-30 06:41:07,NaN,NaN,NaN,NaN,1915-10-20 00:00:00,NaN
last,NaN,NaN,2017-04-18 16:44:26,NaN,NaN,NaN,NaN,2017-04-13 00:00:00,NaN


In [9]:
print(restored_df['critics_pick'].value_counts())
print(restored_df['byline'].value_counts())

False    23766
True      3127
Name: critics_pick, dtype: int64
Bosley Crowther                                                 3022
Stephen Holden                                                  2931
Janet Maslin                                                    2696
Vincent Canby                                                   2601
A O Scott                                                       1790
Mordaunt Hall                                                   1350
Jeannette Catsoulis                                             1180
Manohla Dargis                                                  1142
Neil Genzlinger                                                  545
Howard Thompson                                                  536
Caryn James                                                      527
Frank S Nugent                                                   511
Elvis Mitchell                                                   463
A H Weiler                              

In [10]:
restored_df.select_dtypes(exclude=[np.number]).isnull().sum()

byline                  0
critics_pick            0
date_updated           15
display_title           0
headline                0
mpaa_rating         13377
opening_date         9922
publication_date        0
summary_short        6129
dtype: int64

In [11]:
restored_df['opening_date'].fillna(restored_df['publication_date'], inplace=True)
restored_df

,byline,critics_pick,date_updated,display_title,headline,mpaa_rating,opening_date,publication_date,summary_short
0,Manohla Dargis,True,2017-04-18 16:44:22,The Lost City of Z,Review: Hearts of Darkness and Light in ‘The L...,PG-13,2017-04-14,2017-04-13,Charlie Hunnam stars in this period adventure ...
1,A O Scott,True,2017-04-18 16:44:21,A Quiet Passion,Review: ‘A Quiet Passion’ Poetically Captures ...,PG-13,1492041600000000000,2017-04-13,Cynthia Nixon brings a great 19th-century Amer...
2,A O Scott,True,2017-04-18 16:44:21,Norman,"Review: Richard Gere as ‘Norman,’ an Aspiratio...",R,2017-04-14,2017-04-13,‘Norman: The Moderate Rise and Tragic Fall of ...
3,Jeannette Catsoulis,False,2017-04-18 16:44:26,Tommy's Honour,"Review: In ‘Tommy’s Honour,’ a Family Feud on ...",PG,2017-04-14,2017-04-13,The director Jason Connery’s staid biopic is a...
4,Glenn Kenny,False,2017-04-13 21:58:02,My Entire High School Sinking Into the Sea,"Review: Angst, Jealousy and ‘My Entire High Sc...",PG-13,1492041600000000000,2017-04-13,"Teenage anxiety (voiced by Dash Shaw, Lena Dun..."
5,Jeannette Catsoulis,False,2017-04-18 16:44:25,Mimosas,"Review: In ‘Mimosas,’ a Cryptic Mountain Trek",NaN,1492041600000000000,2017-04-13,"In following nomads on a mission in Morocco, t..."
6,Ken Jaworowski,False,2017-04-18 16:44:25,Maurizio Cattelan: Be Right Back,Review: ‘Maurizio Cattelan: Be Right Back’ Rel...,NaN,2016-04-01,2017-04-13,The documentary explores the artworks and pran...
7,Neil Genzlinger,False,2017-04-18 16:44:24,Little Boxes,"Review: In ‘Little Boxes,’ a Biracial Family M...",NaN,2017-04-14,2017-04-13,A family ventures beyond the bubble of gentrif...
8,Ken Jaworowski,False,2017-04-18 16:44:24,Finding Oscar,"Review: ‘Finding Oscar,’ and Hope, in Massacre...",NaN,2017-04-14,2017-04-13,This documentary looks at a Guatemalan village...
9,Ben Kenigsberg,False,2017-04-18 16:44:23,Chasing Trane: The John Coltrane Documentary,"Review: In ‘Chasing Trane,’ Where’s the Joy of...",NaN,2017-04-14,2017-04-13,John Scheinfeld’s documentary about John Coltr...
